In [1]:
import sys;
import re;
import operator;
import numpy as np;
import math;
from os import listdir;
from keras.utils import np_utils;
from keras.regularizers import l2;
from keras.preprocessing.text import Tokenizer;
from keras.preprocessing.sequence import pad_sequences;
from keras.callbacks import ModelCheckpoint;
from keras.models import Sequential;
from keras.layers import Embedding, LSTM, Dense, Dropout, TimeDistributed, Input;
from keras.optimizers import Adam;

def cleanText(text):
    modifiedString = re.sub("\d", "", text);
    modifiedString = modifiedString.lower();
    modifiedString = modifiedString.replace("\t", " ");
    modifiedString = modifiedString.replace("\n", " ");
    modifiedString = modifiedString.replace('!', " . ");
    modifiedString = modifiedString.replace('"', " ");
    modifiedString = modifiedString.replace('#', " "); 
    modifiedString = modifiedString.replace("'", "'"); 
    modifiedString = modifiedString.replace('(', " "); 
    modifiedString = modifiedString.replace(')', " "); 
    modifiedString = modifiedString.replace(',', " , ");
    modifiedString = modifiedString.replace('-', " "); 
    modifiedString = modifiedString.replace('.', " . "); 
    modifiedString = modifiedString.replace('/', " ");
    modifiedString = modifiedString.replace(':', " ");
    modifiedString = modifiedString.replace(';', " ; "); 
    modifiedString = modifiedString.replace('?', " ? ");
    modifiedString = modifiedString.replace('–', " ");
    modifiedString = modifiedString.replace('—', " ");
    modifiedString = modifiedString.replace('‘', "'"); 
    modifiedString = modifiedString.replace('…', " . ");
    modifiedString = modifiedString.replace('ç', "c");
    modifiedString = modifiedString.replace('é', "e");
    
    return modifiedString;

def generateData(batchSize, dataX, dataY):
    while True:
        randomStart = np.random.randint(len(dataX) - batchSize);
        X = dataX[randomStart:randomStart+batchSize];
        X = np.reshape(X, (batchSize,100));
        Y = dataY[randomStart:randomStart+batchSize];
        Y = np_utils.to_categorical(Y, nb_classes=len(tokenizer.word_index) + 1);
        yield (X, Y);

Using TensorFlow backend.


In [2]:
path = "Stories/";
splitPercentage = 0.2;
files = [f for f in listdir(path) if ("DS_Store" not in f) and ("ipynb" not in f)];
rawData = [];
for f in files:
    text = open(path + f).read();
    text = cleanText(text);
    rawData.append(text);

tokenizer = Tokenizer(filters="");
tokenizer.fit_on_texts(rawData);

wordDimensions = 100;
glovePath = "GloveData/";
embeddingDic = {};
f = open(glovePath + "tingle-vectors-" + str(wordDimensions) + ".txt");
for line in f:
    values = line.split();
    currrentWord = values[0];
    currentVector = np.asarray(values[1:], dtype='float32');
    embeddingDic[currrentWord] = currentVector;
f.close();

dataX = np.zeros((len(tokenizer.word_index) + 1, wordDimensions));
dataY = np.zeros((len(tokenizer.word_index) + 1, 1));
for w, i in tokenizer.word_index.items():
    if w in embeddingDic:
        dataX[i] = embeddingDic[w];
        dataY[i] = [i];

In [28]:
dropOutPercentage = 0.5;
model = Sequential();

model.add(Dense(10 * wordDimensions, input_dim=wordDimensions, W_regularizer=l2(0.01), activation='relu'));
model.add(Dropout(dropOutPercentage));
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'));
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']);

filepath="GloVeToCategorical/converter-tokenizer-{epoch:02d}-{loss:.4f}.hdf5";
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min');
callbacks_list = [checkpoint];

model.fit_generator(generateData(100, dataX, dataY), nb_epoch=100, samples_per_epoch=20000, callbacks=callbacks_list);

Epoch 1/100
20000/20000 [==============================] - 3s - loss: 9.6007 - acc: 0.0263     
Epoch 2/100
20000/20000 [==============================] - 3s - loss: 7.7843 - acc: 0.2320     
Epoch 3/100
20000/20000 [==============================] - 3s - loss: 4.9272 - acc: 0.5513     
Epoch 4/100
20000/20000 [==============================] - 3s - loss: 3.4928 - acc: 0.6849     
Epoch 5/100
20000/20000 [==============================] - 3s - loss: 2.5134 - acc: 0.8019     
Epoch 6/100
20000/20000 [==============================] - 3s - loss: 1.9752 - acc: 0.8582     
Epoch 7/100
20000/20000 [==============================] - 3s - loss: 1.6267 - acc: 0.9042     
Epoch 8/100
20000/20000 [==============================] - 3s - loss: 1.5192 - acc: 0.9102     
Epoch 9/100
20000/20000 [==============================] - 3s - loss: 1.4188 - acc: 0.9234     
Epoch 10/100
20000/20000 [==============================] - 3s - loss: 1.1682 - acc: 0.9527     
Epoch 11/100
20000/20000 [=============

KeyboardInterrupt: 

In [42]:
model = Sequential();

noisyX = dataX + np.random.normal(0, 0.3, wordDimensions);

model.add(Dense(10 * wordDimensions, input_dim=wordDimensions, activation='relu'));
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'));
model.load_weights("GloVeToCategorical/converter-tokenizer-0.3000.hdf5");
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']);
f = open("GloVeToCategorical/model.json", "w");
f.write(model.to_json());
f.close();

print(model.evaluate_generator(generateData(100, noisyX, dataY), len(dataX)));

[0.2558587496448308, 0.971199978351593]


In [3]:
f = open("GloVeToCategorical/reverseDic.txt", "w");
for w, i in tokenizer.word_index.items():
    f.write(str(i) + ", " + w + "\n");
f.close();

FileNotFoundError: [Errno 2] No such file or directory: 'GloVeToCategorical/reverseDic.txt'